In [4]:
import math
import time
import wandb
import rpnn
import pprint

ToggleButton(value=False, description='Toggle Rendering')

In [ ]:
WANDB_PROJECT = 'function-approximation'
wandb.login()

In [ ]:
sweep_config = {
    'method': 'random'
}

In [ ]:
metric = {
    'name': 'squared_error',
    'goal': 'minimize'
}

sweep_config['metric'] = metric

In [ ]:
# numerical precision params
parameters_dict = {
    'fp_precision': {
        'distribution': 'q_uniform',
        'q': 1,
        'min': 4,
        'max': 24,
    },
    
    'fp_emax': {
        'distribution': 'q_uniform',
        'q': 1,
        'min': 4,
        'max': 127,
    },
    
    'fp_subnormal': {
        'value': 'CPFLOAT_SUBN_USE'
    },
    
    'fp_round': {
        'values': ['CPFLOAT_RND_SP',
                   'CPFLOAT_RND_SE']
    },
    
    'fp_flip': {
        'value': 'CPFLOAT_NO_SOFTERR'
    },
    
    'fp_explim': {
        'value': 'CPFLOAT_EXPRANGE_TARG'
    },
    
    'fp_p': {
        'value': 0.0
    },
}


In [ ]:
# reinforcement learning (RL) params
parameters_dict = {
    'epsilon0' : {
        'value' = 1.0
    },
    'epsilon1' : {
        'value' = 0.1
    },
    'exploration_steps' : {
        'value' = 1000000
    },
    'epsilon' : {
        'value' = 1.0
    },
    
    'gamma' : {
        'value' = 0.99
    },
    'alpha' : {
        'value' = 1.0
    },
    
    'target_update_frequency' : {
        'value' = 10000
    },
    'replay_start_size' : {
        'value' = 50000
    },
    
    'action_repeat' : {
        'value' = 1
    }
}

In [ ]:
# neural network params
parameters_dict.update({
    'layer_sizes': {
        'value': (4, 16, 16, 16, 2)
    },
    'epochs': {
        'value': 50000
    },
    'minibatch_size': {
        'value': 32
    },
    'learning_rate': {
        'value': 0.1
    },
    'weight_decay': {
        'value': 0.001
    },
    'dropout_keep_p': {
        'value': 1
    },
    'momentum_coefficient': {
        'value': 0
    },
    'loss': {
        'value': 'mean_squared_error'
    }
})

In [ ]:
sweep_config['parameters'] = parameters_dict

In [ ]:
def train():
    run = wandb.init()
    config = run.config
    
    squared_error = rpnn.matrix_factorization(features=config["features"],
        epochs=config["epochs"],
        non_negative=config["non_negative"],
        learning_rate=config["learning_rate"],
        weight_decay=config["weight_decay"],
        dropout_keep_p=config["dropout_keep_p"],
        momentum_coefficient=config["momentum_coefficient"],
        fp_precision=config["fp_precision"],
        fp_emax=config["fp_emax"],
        fp_subnormal=config["fp_subnormal"],
        fp_round=config["fp_round"],
        fp_flip=config["fp_flip"],
        fp_p=config["fp_p"],
        fp_explim=config["fp_explim"]
    )
    
    wandb.log({'squared_error': squared_error}, commit=False, step=0)
        
    run.finish()

In [ ]:
sweep_id = wandb.sweep(sweep_config, project=WANDB_PROJECT)
#sweep_id = "jneupcgr"

wandb.agent(sweep_id=sweep_id, project=WANDB_PROJECT, function=train, count=100)